# Image Mosaicing

1. Use any feature detector and descriptor (e.g. SIFT) to find matches between two
partially overlapping images. You can use inbuilt functions for this.
2. Estimate the homography matrix between the two images robustly.
3. Transform one of the images to the other’s reference frame using the homography
matrix.
4. Stitch the two images together.
5. Repeat this for multiple images to produce a singly mosaic/panorama.
6. Demonstrate the results on different scenes from the given data.
7. Additionally, capture a set of overlapping images of a scene with your camera and
report the results on the same.
8. BONUS: Think of an algorithm which can stitch images given in any order without
human intervention. Modify your existing code accordingly.

In [79]:
#Import necessary libraries
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [75]:
class Mosaicing:

    def stich_images(self, images, lowe_ratio=0.5, max_Threshold=2.0,match_status=False):

        #detect the features and keypoints from SIFT
        (imageB, imageA) = images
        (KeypointsA, features_of_A) = self.Find_Feature_And_KeyPoints(imageA)
        (KeypointsB, features_of_B) = self.Find_Feature_And_KeyPoints(imageB)

        #got the valid matched points
        Values = self.matchKeypoints(KeypointsA, KeypointsB,features_of_A, features_of_B, lowe_ratio, max_Threshold)

        if Values is None:
            return None

        #to get perspective of image using computed homography
        (matches, Homography, status) = Values
        result_image = self.getwarp_perspective(imageA,imageB,Homography)
        result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

        # check to see if the keypoint matches should be visualized
        if match_status:
            vis = self.draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches,status)

            return (result_image, vis)

        return result_image

    def getwarp_perspective(self,imageA,imageB,Homography):
        val = imageA.shape[1] + imageB.shape[1]
        result_image = cv2.warpPerspective(imageA, Homography, (val , imageA.shape[0]))

        return result_image

    def Find_Feature_And_KeyPoints(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect and extract features from the image
        descriptors = cv2.xfeatures2d.SIFT_create()
        (Keypoints, features) = descriptors.detectAndCompute(image, None)

        Keypoints = np.float32([i.pt for i in Keypoints])
        return (Keypoints, features)

    def get_Allpossible_Match(self,featuresA,featuresB):

        # compute the all matches using euclidean distance and opencv provide
        #DescriptorMatcher_create() function for that
        match_instance = cv2.DescriptorMatcher_create("BruteForce")
        All_Matches = match_instance.knnMatch(featuresA, featuresB, 2)

        return All_Matches

    def All_validmatches(self,AllMatches,lowe_ratio):
        #to get all valid matches according to lowe concept..
        valid_matches = []

        for val in AllMatches:
            if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
                valid_matches.append((val[0].trainIdx, val[0].queryIdx))

        return valid_matches

    def Compute_Homography(self,pointsA,pointsB,max_Threshold):
        #to compute homography using points in both images

        (H, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC, max_Threshold)
        return (H,status)

    def matchKeypoints(self, KeypointsA, KeypointsB, featuresA, featuresB,lowe_ratio, max_Threshold):

        AllMatches = self.get_Allpossible_Match(featuresA,featuresB);
        valid_matches = self.All_validmatches(AllMatches,lowe_ratio)

        if len(valid_matches) > 4:
            # construct the two sets of points
            pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
            pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

            (Homograpgy, status) = self.Compute_Homography(pointsA, pointsB, max_Threshold)

            return (valid_matches, Homograpgy, status)
        else:
            return None

    def get_image_dimension(self,image):
        (h,w) = image.shape[:2]
        return (h,w)

    def get_points(self,imageA,imageB):

        (hA, wA) = self.get_image_dimension(imageA)
        (hB, wB) = self.get_image_dimension(imageB)
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB

        return vis


    def draw_Matches(self, imageA, imageB, KeypointsA, KeypointsB, matches, status):

        (hA,wA) = self.get_image_dimension(imageA)
        vis = self.get_points(imageA,imageB)

        # loop over the matches
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            if s == 1:
                ptA = (int(KeypointsA[queryIdx][0]), int(KeypointsA[queryIdx][1]))
                ptB = (int(KeypointsB[trainIdx][0]) + wA, int(KeypointsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        return vis
    

In [76]:
def resize(src, width=None, height=None, inter=cv2.INTER_AREA):
        
        #Utility Function: Resizes Image to new width and height
        #Mention new width and height, else None expected.
        #Default interpolation - INTER_AREA
        
        print("Resizing... ")
        # set dimension to None, to return if no dimension specified
        dim = None
        (h, w) = src.shape[:2] # take height and width

        # handle None values in width and height
        if width is None and height is None:
            return src 

        # to resize, aspect ratio has to be kept in check
        # for no distortion of the shape of the imag3
        if width is None:
            # calculate aspect ratio changed according to new height
            ratio = height / float(h)
            # change width according to the ratio
            dim = (int(w * ratio), height)
        else:
            # calculate aspect ratio changed according to new width
            ratio = width/float(w)
            dim = (width, int(h * ratio))
        # apply resizing using calculated dimensions 
        result = cv2.resize(src, dim, interpolation=inter)
    
        return result

In [81]:
path  = "images/Image Mosaicing/"  # Path to input images
# Enter image names to stich in the images_list as below
images_list = ["6_1.jpg", "6_2.jpg", "6_3.jpg","6_4.jpg", "6_5.jpg", "6_6.jpg"]
images_list = [path + x for x in images_list]
no_of_images = len(images_list)
print('number of images', no_of_images)
images = []
#print(images_list)
for i in range(no_of_images):
    images.append(cv2.imread(images_list[i])) # read images from given path
print('images read')
#print(images[0])
#We need to modify the image resolution and keep our aspect ratio  

for i in range(no_of_images):
    images[i] = resize(images[i], width=400)

for i in range(no_of_images):
    images[i] = resize(images[i], height=400)
count = 0
image_mosaicing = Mosaicing()
if no_of_images==2:
    #(result) = image_mosaicing.stich_images([images[0], images[1]], match_status=True)
    (result, matched_points) = image_mosaicing.stich_images([images[0], images[1]], match_status=True)
else:
    print('no_of_images', no_of_images)
    '''
    (result, matched_points) = image_mosaicing.stich_images([images[no_of_images-2], images[no_of_images-1]], match_status=True)
    for i in range(no_of_images - 2):
        print('i value',i)
        (result, matched_points) = image_mosaicing.stich_images([images[no_of_images-i-3],result], match_status=True)
    '''
    (result, matched_points) = image_mosaicing.stich_images([images[0], images[1]], match_status=True)
     
    #cv2.imwrite("Image6.png",result)
    count = 2
    while count < no_of_images:
        print('count',count)
        (result, matched_points) = image_mosaicing.stich_images([images[count], result], match_status=True)
         
        count = count + 1
    
#to write the images
cv2.imwrite("Matched_points6-3.png",matched_points)
cv2.imwrite("Image_Mosaicing6-3.png",result)




number of images 6
images read
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
Resizing... 
no_of_images 6
count 2
count 3
count 4
count 5


True

# Results

All input images are tested with the code. Results are captured in

Image_Mosaicing1.png
Image_Mosaicing2.png
Image_Mosaicing3.png
Image_Mosaicing4.png
Image_Mosaicing5.png
Image_Mosaicing6-3.png (These overlapping pictures are taken from my camera. These images are captured from closer and from farther distance. It has been observed that valid matches are stiched into panorama with highest key matches.)

Keypoints are captured here ->
Matched_points1.png
Matched_points2.png
Matched_points3.png
Matched_points4.png
Matched_points5.png
Matched_points6-3.png

